In [513]:
from astropy.io import ascii
from astropy.table import Table

In [514]:
# w = open('/Users/bmmorris/Desktop/targets.txt', 'w') 
# w.write('\n'.join(gaia['Target'].data.tolist() + ['GJ 4099', 'EPIC 211928486', 'EPIC 211966629']))
# w.close()

In [515]:
from json import load

star_key = load(open('stars.json'))

spot_stars = Table(rows=[[star, star_key[star][1][0]] for star in star_key], names=['Target', 'Spot'])

In [516]:
data_start = 3
gaia = ascii.read('temperatures/gaia2.tsv', #names='Target,Gmag,Teff_Gaia,Teff_Gaia_lower,Teff_Gaia_upper'.split(','), 
                  data_start=data_start, delimiter='\t')
# spocs2005 = ascii.read('temperatures/spocs2005_temps.tsv', names='Target, Teff_spocs2005'.split(', '), 
#                        data_start=data_start, delimiter='\t')
# spocs2016 = ascii.read('temperatures/spocs2016_temps.tsv', names='Target, Teff_spocs2016'.split(', '), 
#                        data_start=data_start, delimiter='\t')

In [517]:
gb = gaia.group_by('_1')

In [518]:
from astropy.table import vstack

gaia_dedup = Table(gb.groups[0])

for i in np.arange(1, len(gb.groups)): 
    group = gb.groups[i]
    gaia_dedup.add_row(group[np.argmin(group['Gmag'])])
    #gaia_dedeup = vstack([gaia_dedup, group[np.argmin(group['Gmag'])]], join_type='inner')

In [519]:
remove_cols = ['_r', 'RA_ICRS', 'e_RA_ICRS', 'DE_ICRS', 'e_DE_ICRS', 'Source', 'Plx', 'e_Plx', 
               'pmRA', 'e_pmRA', 'pmDE', 'e_pmDE', 'Dup', 'FG', 'e_FG', 'Gmag', 'e_Gmag', 'FBP', 
               'e_FBP', 'BPmag', 'e_BPmag', 'FRP', 'e_FRP', 'RPmag', 'e_RPmag', 'BP-RP', 'RV', 
               'e_RV', 'AG', 'E(BP-RP)', 'Rad', 'Lum', 'DR2Name', 'SolID', 'RandomI', 'Epoch', 
               'RPlx', 'RADEcor', 'RAPlxcor', 'RApmRAcor', 'RApmDEcor', 'DEPlxcor', 'DEpmRAcor', 
               'DEpmDEcor', 'PlxpmRAcor', 'PlxpmDEcor', 'pmRApmDEcor', 'NAL', 'NAC', 'NgAL', 
               'NbAL', 'gofAL', 'chi2AL', 'epsi', 'sepsi', 'Solved', 'APF', 'WAL', 'pscol', 
               'e_pscol', 'fvarpi', 'MatchObsA', 'Nper', 'amax', 'type', 'MatchObs', 'o_Gmag', 
               'RFG', 'o_BPmag', 'RFBP', 'o_RPmag', 'RFRP', 'E(BR/RP)', 'Mode', 'BP-G', 'G-RP', 
               'o_RV', 'Tefftemp', 'loggtemp', '[Fe/H]temp', 'Var', 'GLON', 'GLAT', 'ELON', 'ELAT', 
               'fPriam', 'b_AG', 'B_AG', 'b_E(BP-RP)', 'B_E(BP-RP)', 'fFLAME', 'b_Rad', 'B_Rad', 
               'b_Lum', 'B_Lum', 'RAJ2000', 'e_RAJ2000', 'DEJ2000', 'e_DEJ2000']
# 'Teff', 'b_Teff', 'B_Teff', 

gaia_dedup.remove_columns(remove_cols)

In [520]:
gaia_dedup.rename_column('_1', 'Target')

In [521]:
gaia_dedup.rename_column('b_Teff', 'Teff_Gaia_lower')
gaia_dedup.rename_column('B_Teff', 'Teff_Gaia_upper')
gaia_dedup.rename_column('Teff', 'Teff_Gaia')

In [522]:
import numpy as np
gaia_teffs = [gaia_dedup['Teff_Gaia'], 
              gaia_dedup['Teff_Gaia']-gaia_dedup['Teff_Gaia_lower'], 
              gaia_dedup['Teff_Gaia_upper']-gaia_dedup['Teff_Gaia']]

gaia_formatted = []
for mid, lower, upper in Table(gaia_teffs, names=['Mid', 'Lower', 'Upper']):
    if isinstance(mid, float):
        gaia_formatted.append('${0:.0f}^{{+{1:.0f}}}_{{-{2:.0f}}}$'.format(mid, upper, lower))
    else: 
        gaia_formatted.append('')

In [523]:
from astropy.table import Column
gaia_dedup.remove_columns(['Teff_Gaia', 'Teff_Gaia_lower', 'Teff_Gaia_upper'])
gaia_dedup.add_column(Column(gaia_formatted, '$T_\mathrm{eff,Gaia}$'))
gaia_dedup['Target'] = [s.strip().replace(' ', '') for s in list(gaia_dedup['Target'])]

In [524]:
gaia_spot = join(gaia_dedup, spot_stars, join_type='outer')

gaia_spot.add_index('Target')

spot_temps = []
for s in gaia_spot['Spot']:
    if isinstance(s, str) and s in gaia_spot['Target']:
        spot_temps.append(gaia_spot.loc[s]['$T_\mathrm{eff,Gaia}$'])
    else: 
        spot_temps.append("")
        
gaia_spot.add_column(Column(spot_temps, '$T_S$'))
gaia_spot.remove_column('Spot')

In [525]:
import sys
gaia_spot.write(sys.stdout, format='latex')

\begin{table}
\begin{tabular}{ccc}
Target & $T_\mathrm{eff,Gaia}$ & $T_S$ \\
51Peg & $5830^{+214}_{-55}$ &  \\
EKDra & $5584^{+115}_{-193}$ & $5253^{+61}_{-89}$ \\
EPIC211928486 & $4682^{+149}_{-250}$ & $4182^{+260}_{-132}$ \\
EPIC211966629 & $4922^{+83}_{-270}$ & $4182^{+260}_{-132}$ \\
EQVir & $4433^{+82}_{-106}$ & $3846^{+181}_{-190}$ \\
GJ4099 & $3846^{+181}_{-190}$ &  \\
GJ702B & $5305^{+341}_{-245}$ & $4851^{+13}_{-75}$ \\
GJ705 & $4465^{+102}_{-66}$ &  \\
GJ9781A & $4484^{+95}_{-110}$ &  \\
HAT-P-11 & $4757^{+154}_{-111}$ &  \\
HD10697 & $5654^{+36}_{-56}$ &  \\
HD110833 & $5025^{+50}_{-82}$ &  \\
HD113827 & $4610^{+459}_{-191}$ & $3882^{+163}_{-82}$ \\
HD122120 & $4469^{+253}_{-144}$ &  \\
HD127506 & $4711^{+55}_{-83}$ & $4465^{+102}_{-66}$ \\
HD129333 & $5584^{+115}_{-193}$ &  \\
HD134319 & $5636^{+100}_{-54}$ & $5253^{+61}_{-89}$ \\
HD14039 & $5072^{+98}_{-73}$ &  \\
HD145675 & $5282^{+251}_{-73}$ &  \\
HD145742 & $4851^{+13}_{-75}$ &  \\
HD148467 & $4474^{+153}_{-201}$ &  \\

In [526]:
sinds = Table.read('/Users/bmmorris/git/arces_hk/freckles.tex')
sinds.sort('Target')

In [527]:
sinds['Target'] = [s.strip().replace(' ', '') for s in sinds['Target']]

In [528]:
sinds.pprint(max_lines=1000)

    Target      Sp. Type         $S$       $\log R^\prime_\mathrm{HK}$ Active? 
------------- ------------ --------------- --------------------------- --------
        51Peg         G2IV $0.15 \pm 0.01$                        -4.9 Inactive
        EKDra        G1.5V $0.58 \pm 0.02$                       -3.91   Active
        EQVir         K5Ve $3.00 \pm 0.12$                       -3.65   Active
       GJ4099       M2.0Ve $0.87 \pm 0.05$                       -4.67 Inactive
       GJ702B          K4V $0.84 \pm 0.04$                       -3.61   Active
        GJ705           K2 $0.12 \pm 0.01$                       -5.44 Inactive
      GJ9781A           K7 $0.99 \pm 0.04$                       -4.46   Active
     HAT-P-11          K4V $0.53 \pm 0.02$                       -4.46   Active
      HD10697         G3Va $0.14 \pm 0.01$                       -5.01 Inactive
     HD110833          K3V $0.30 \pm 0.01$                       -4.52 Inactive
     HD113827          K4V $1.03 \pm 0.0

In [529]:
from astropy.table import hstack

temps_sinds = join(gaia_spot, sinds, join_type='outer', keys='Target')

In [530]:
temps_sinds.pprint(max_lines=1000, max_width=1000)

    Target    $T_\mathrm{eff,Gaia}$        $T_S$           Sp. Type         $S$       $\log R^\prime_\mathrm{HK}$ Active? 
------------- --------------------- -------------------- ------------ --------------- --------------------------- --------
        51Peg   $5830^{+214}_{-55}$                              G2IV $0.15 \pm 0.01$                        -4.9 Inactive
        EKDra  $5584^{+115}_{-193}$   $5253^{+61}_{-89}$        G1.5V $0.58 \pm 0.02$                       -3.91   Active
EPIC211928486  $4682^{+149}_{-250}$ $4182^{+260}_{-132}$           --              --                          --       --
EPIC211966629   $4922^{+83}_{-270}$ $4182^{+260}_{-132}$           --              --                          --       --
        EQVir   $4433^{+82}_{-106}$ $3846^{+181}_{-190}$         K5Ve $3.00 \pm 0.12$                       -3.65   Active
       GJ4099  $3846^{+181}_{-190}$                            M2.0Ve $0.87 \pm 0.05$                       -4.67 Inactive
       GJ702B  $

In [531]:
temps_sinds.sort('$\log R^\prime_\mathrm{HK}$')
temps_sinds.remove_column('Active?')

In [532]:
names = list(star_key.keys())
comp_stars = Table([names, ["{0}, {1}".format(star_key[star][0][0], star_key[star][1][0]) for star in names]], names=['Target', 'Comp.'])

In [533]:
fs_table = Table.read('f_S.tex')
fs_table['Target'] = [t.replace(' ', '') for t in fs_table['Target'] ]

In [534]:
full_table = join(temps_sinds, fs_table, join_type='outer', keys='Target')
full_table.sort('$\log R^\prime_\mathrm{HK}$')

In [535]:
# full_table.remove_row(-1)
full_table.write('megatable.tex', format='aastex', caption="\label{tab:megatable}")

In [536]:
full_table.pprint(max_lines=1000, max_width=1000)

    Target    $T_\mathrm{eff,Gaia}$        $T_S$           Sp. Type         $S$       $\log R^\prime_\mathrm{HK}$      $f_S$     
------------- --------------------- -------------------- ------------ --------------- --------------------------- ---------------
       HD5857    $4712^{+67}_{-71}$                                G5 $0.11 \pm 0.01$                       -5.53              --
        GJ705   $4465^{+102}_{-66}$                                K2 $0.12 \pm 0.01$                       -5.44              --
       HD6497    $4514^{+82}_{-71}$                          K2III-IV $0.11 \pm 0.01$                       -5.41              --
      HD89744    $6239^{+23}_{-84}$                               F7V $0.14 \pm 0.01$                       -5.11              --
     HD145742    $4851^{+13}_{-75}$                                K0 $0.15 \pm 0.01$                       -5.09              --
      HD10697    $5654^{+36}_{-56}$                              G3Va $0.14 \pm 0.01$     

In [537]:
!cat megatable.tex

\begin{deluxetable}{ccccccc}
\tablecaption{\label{tab:megatable}}
\tablehead{\colhead{Target} & \colhead{$T_\mathrm{eff,Gaia}$} & \colhead{$T_S$} & \colhead{Sp. Type} & \colhead{$S$} & \colhead{$\log R^\prime_\mathrm{HK}$} & \colhead{$f_S$}}
\startdata
HD5857 & $4712^{+67}_{-71}$ &  & G5 & $0.11 \pm 0.01$ & -5.53 &  \\
GJ705 & $4465^{+102}_{-66}$ &  & K2 & $0.12 \pm 0.01$ & -5.44 &  \\
HD6497 & $4514^{+82}_{-71}$ &  & K2III-IV & $0.11 \pm 0.01$ & -5.41 &  \\
HD89744 & $6239^{+23}_{-84}$ &  & F7V & $0.14 \pm 0.01$ & -5.11 &  \\
HD145742 & $4851^{+13}_{-75}$ &  & K0 & $0.15 \pm 0.01$ & -5.09 &  \\
HD10697 & $5654^{+36}_{-56}$ &  & G3Va & $0.14 \pm 0.01$ & -5.01 &  \\
HD221639 & $5102^{+139}_{-106}$ &  & K1V & $0.15 \pm 0.01$ & -4.96 &  \\
HD210277 & $5535^{+42}_{-48}$ &  & G8V & $0.15 \pm 0.01$ & -4.94 &  \\
HD145675 & $5282^{+251}_{-73}$ &  & K0V & $0.16 \pm 0.01$ & -4.94 &  \\
HD86728 & $5832^{+251}_{-207}$ &  & G3VaHdel1 & $0.15 \pm 0.01$ & -4.93 &  \\
HD34411 & $5979^{+178}_{-140}$ &